[날짜 구현](https://jsikim1.tistory.com/144)  
[날짜 계산](https://jsikim1.tistory.com/143)  
[날짜, 시간을 문자열로 변환](https://jsikim1.tistory.com/216)  
[xml to dataframe (1)](https://w0lfydocoding.tistory.com/46)  
[xml to dataframe (2)](https://rfriend.tistory.com/477)

# XML to DataFrame (OPpen API 데이터 활용)

## 0. 테스트 예시
- 서울열린데이터광장 → 장애인콜시스템 → 2022년 8월 15일 정보 가져오기

In [1]:
import requests
from bs4 import BeautifulSoup
import urllib.request as urlreq
import os.path
import pandas as pd
from datetime import datetime, timedelta

res = requests.get("http://openapi.seoul.go.kr:8088/745a64744764756434305a43436175/xml/disabledCalltaxi/1/700/20220815")
soup = BeautifulSoup(res.text, 'lxml')
info = []

for location in soup.find_all('item') :
    no = location.find('no').string  
    cartype = location.find('cartype').string 
    receipttime = location.find('receipttime').string  
    settime = location.find('settime').string 
    ridetime = location.find('ridetime').string
    startpos1 = location.find('startpos1').string
    startpos2 = location.find('startpos2').string
    endpos1 = location.find('endpos1').string
    endpos2 = location.find('endpos2').string
    
    info.append({'no': no, 'cartype': cartype, 'receipttime': receipttime, 'settime': settime, 'ridetime': ridetime, 
                 'startpos1': startpos1, 'startpos2': startpos2, 'endpos1': endpos1, 'endpos2': endpos2})

columns=['no', 'cartype', 'receipttime', 'settime', 'ridetime', 'startpos1', 'startpos2', 'endpos1', 'endpos2']
df = pd.DataFrame(info, columns = columns)
df.head()

,no,cartype,receipttime,settime,ridetime,startpos1,startpos2,endpos1,endpos2
0,6774,중형승합,2022-08-15 오전 12:09:00,2022-08-15 오전 1:07:46,2022-08-15 오전 1:32:14,서대문구,신촌동,서대문구,신촌동
1,7857,중형승합,2022-08-15 오전 12:09:00,2022-08-15 오전 1:19:40,2022-08-15 오전 1:50:32,용산구,남영동,강서구,방화제2동
2,7548,중형승합,2022-08-15 오전 12:26:00,2022-08-15 오전 1:36:30,2022-08-15 오전 1:49:34,강동구,명일제2동,서초구,내곡동
3,821,중형승합,2022-08-15 오전 12:38:00,2022-08-15 오전 2:55:25,2022-08-15 오전 3:24:46,성북구,월곡제2동,성북구,길음제2동
4,8029,중형 승합,2022-08-15 오전 12:46:00,2022-08-15 오전 1:31:29,2022-08-15 오전 2:03:53,강남구,수서동,강서구,화곡제1동


## 1. 필요한 라이브러리 불러오기

In [2]:
import requests
from bs4 import BeautifulSoup
import urllib.request as urlreq
import os.path
import pandas as pd
from datetime import datetime, timedelta

## 2. 날짜 리스트 만들기(21.04.28 ~ 22.08.15)

In [2]:
daytime = datetime.strptime("20210428","%Y%m%d")
datetime_lst = []

while True:
    datetime_lst.append(daytime.strftime("%Y%m%d"))
    if daytime == daytime.strptime("20220815","%Y%m%d"):
        break
    daytime = daytime + timedelta(days=1)

## 3. api url 리스트 만들기(21.04.28 ~ 22.08.15)

In [3]:
api_urls_lst = []
api = "http://openapi.seoul.go.kr:8088/745a64744764756434305a43436175/xml/disabledCalltaxi/1/700/"
for i in range(len(datetime_lst)):
    api_urls_lst.append(api + datetime_lst[i])

## 4. for문을 활용한 DataFrame 생성

In [15]:
frist_df = pd.DataFrame() # 빈 데이터프레임 생성
final_df = pd.DataFrame() # 빈 데이터프레임 생성
columns=['no', 'cartype', 'receipttime', 'settime', 'ridetime', 'startpos1', 'startpos2', 'endpos1', 'endpos2']

for api in range(350, 400):
    res = requests.get(api_urls_lst[api])
    soup = BeautifulSoup(res.text, 'lxml')
    info = []

    for location in soup.find_all('item') :
        no = location.find('no').string  
        cartype = location.find('cartype').string 
        receipttime = location.find('receipttime').string  
        settime = location.find('settime').string 
        ridetime = location.find('ridetime').string
        startpos1 = location.find('startpos1').string
        startpos2 = location.find('startpos2').string
        endpos1 = location.find('endpos1').string
        endpos2 = location.find('endpos2').string

        info.append({'no': no, 'cartype': cartype, 'receipttime': receipttime, 'settime': settime, 'ridetime': ridetime, 
                     'startpos1': startpos1, 'startpos2': startpos2, 'endpos1': endpos1, 'endpos2': endpos2})
    frist_df = pd.DataFrame(info, columns = columns)
    final_df = pd.concat([final_df, frist_df])

final_df.head()

,no,cartype,receipttime,settime,ridetime,startpos1,startpos2,endpos1,endpos2
0,8279,중형 승합,2022-04-13 오전 12:19:54,2022-04-13 오전 12:36:54,2022-04-13 오전 1:01:07,용산구,원효로제1동,관악구,청림동
1,8215,중형승합,2022-04-13 오전 12:26:00,2022-04-13 오전 12:30:04,2022-04-13 오전 12:50:02,동대문구,회기동,성동구,금호1가동
2,7640,중형승합,2022-04-13 오전 12:35:11,2022-04-13 오전 1:04:57,2022-04-13 오전 1:18:30,중랑구,중화제1동,성북구,길음제1동
3,7663,중형승합,2022-04-13 오전 12:36:00,2022-04-13 오전 1:08:35,2022-04-13 오전 1:25:53,강서구,가양제2동,광명시,하안3동
4,4974,중형 승합,2022-04-13 오전 12:38:00,2022-04-13 오전 1:09:19,2022-04-13 오전 1:38:10,중구,회현동,강북구,수유제3동


In [16]:
final_df.tail()

,no,cartype,receipttime,settime,ridetime,startpos1,startpos2,endpos1,endpos2
1917,809,중형승합,2022-06-01 오후 11:35:18,2022-06-01 오후 11:40:11,2022-06-01 오후 11:59:12,서대문구,신촌동,구로구,오류제2동
1918,1554,중형 승합,2022-06-01 오후 11:40:00,2022-06-01 오후 11:46:11,2022-06-01 오후 11:55:42,도봉구,창제4동,성북구,길음제1동
1919,8069,중형 승합,2022-06-01 오후 11:46:04,2022-06-01 오후 11:48:21,2022-06-01 오후 11:58:52,금천구,독산제1동,영등포구,대림제2동
1920,8075,중형 승합,2022-06-01 오후 11:49:00,2022-06-02 오전 12:04:55,2022-06-02 오전 12:26:10,강서구,공항동,구로구,구로제2동
1921,8229,중형승합,2022-06-01 오후 11:49:05,2022-06-01 오후 11:50:05,2022-06-02 오전 12:16:03,중구,회현동,관악구,성현동


In [17]:
final_df.shape

(187733, 9)

## 5. 생성된 DataFrame 저장

In [18]:
final_df.to_excel("data/장애인콜시스템(22.04.13 ~ 22.06.01).xlsx")